In [13]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader,  Subset, random_split, Dataset

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


In [15]:

# Define the hyperparameters
input_size = 1  # Number of features (e.g., previous stock prices)
hidden_size = 64  # Number of LSTM units
num_layers = 2  # Number of LSTM layers
output_size = 1  # Number of output predictions (e.g., predicted stock price)
epochs = 100



In [16]:
#Data


train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)


NameError: name 'train_data' is not defined

In [ ]:
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMRegressor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Get the last time step output and pass it through the linear layer
        return out

In [ ]:
model = LSTMRegressor(input_size, hidden_size, num_layers, output_size)
model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0.0
    for inputs, targets in train_dataloader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    return model, average_loss

In [ ]:
def test_model(model, val_dataloader, device):
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        for inputs, targets in test_dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
        
        average_test_loss = test_loss / len(test_dataloader)
        return average_test_loss

In [ ]:
def save_only_highest(current_accuracy, epoch):
    model_path = f"../models/MoneyBrinterBRR/acc:{current_accuracy:.2f}| epoch:{epoch}"
    filenames = os.listdir(model_path)
    highest_accuracy = max([float(str.split(filename, ':')[1]) for filename in filenames])

    if current_accuracy > highest_accuracy:
        filepath_with_highest_accuracy = f"../models/dogs-cats/dogs-cats-cnn.pth-accurracy-{highest_accuracy}"
        torch.save(model.state_dict(), model_path)
        os.remove(filepath_with_highest_accuracy) 
        print(f"Saved the highest accuracy model to {model_path}, with accuracy {current_accuracy:.2f}")
        

In [ ]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    model = train_model(model, train_dataloader, loss_fn, optimizer, device)
    accuracy = test_model(model, val_dataloader, device)
    save_only_highest(accuracy, epoch)
    print(f"Epoch [{epoch+1}/{epochs}], Test Accuracy: {accuracy:.2f}%")
print("Done!")